# Displaying member similarity
> GUI to explore most similar members

In [ ]:
%load_ext autoreload
%autoreload 2

## Setup

In [ ]:
import pandas as pd

import bundestag.logging as logging
import bundestag.paths as paths
from bundestag.gui import MdBGUI, PartyGUI

logger = logging.logger
logger.setLevel("DEBUG")

_paths = paths.get_paths("../data")
_paths

In [ ]:
file = _paths.preprocessed_bundestag / "bundestag.de_votes.parquet"
file

In [ ]:
df = pd.read_parquet(file)

## Politician (MdB) GUI

In [ ]:
mdb = MdBGUI(df)

In [ ]:
mdb.render()

## Party GUI

In [ ]:
party = PartyGUI(df)

In [ ]:
party.render()